In [17]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

# from attention_utils import get_activations, get_data_recurrent


In [18]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [19]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [20]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [21]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims, tokenize=True):
    if not tokenize:
        words = textline.split()
    else:
        words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [22]:
def sequential_generator(filename, 
                         batch_size, 
                         labels2Idx:'dict to make output labels',
                         check:'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None,
                         tokenize:'specify if using twitter tokenzor to preprocess lines'=False, 
                        ):    
    
    f = open(filename)
    n_labels = len(labels2Idx)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#         print(type(labels2Idx))
        batch_labels = np.zeros((batch_size, len(labels2Idx)))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims, tokenize= tokenize)
            if len(labels2Idx)==2:
                batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            else:
                batch_labels[i] = to_categorical(labels2Idx[data[1]], n_labels)
        yield ([batch_features_ft], batch_labels)

In [23]:
def train_dev_sentences(filetrain,filedev, check:'to check if lines of file are all same lenght after separating by tab'):
    labels2Idx = {}
    train_lines = [line.strip().split("\t") for line in open(filetrain) if len(line.strip().split('\t'))==check]
    dev_lines = [line.strip().split("\t") for line in open(filedev) if len(line.strip().split('\t'))==check]

    train_sentences = [x[0] for x in train_lines]
    for dataset in [train_lines, dev_lines]:
        for line in dataset:
            label = line[1]
            if label not in labels2Idx.keys():
                labels2Idx[label]= len(labels2Idx)
                
#     train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]
    train_labels = [labels2Idx[x[1]] for x in train_lines]
    dev_sentences = [x[0] for x in dev_lines]
#     dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]
    dev_labels = [labels2Idx[x[1]] for x in dev_lines]
    return (train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx)


In [43]:
file_train = '/home/jindal/notebooks/twitter_data/twitter_classes_500k_train.csv'
file_dev = '/home/jindal/notebooks/twitter_data/twitter_classes_500k_dev.csv'
train_sentences, train_labels, dev_sentences, dev_labels, labels2Id = train_dev_sentences(filetrain=file_train,filedev=file_dev, check=2)

In [44]:
print(len(dev_sentences))

100000


In [45]:
dev_labels[:10]

[421, 10, 274, 329, 134, 462, 369, 91, 273, 362]

In [46]:
print(len(dev_labels))

100000


In [47]:
n_labels = 500

In [24]:
def compile_model(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    # model_concatenated = Dropout(0.8)(model_concatenated)
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
#     new_model.summary()
    return new_model

In [71]:
print(dev_sentences[1])

RT USER_MENTION : URL / i2a32rRGUt


In [8]:
def test_model(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels_earlier:'number of original labels if loading a pretrained model',
               filename_to_save_weigths,
               batch_size, 
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               labels2Idx,
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None,
                ):
    
#     f = open(filename_to_log,"w")
    
    max_f1=0
    max_p=0
    max_r=0
    max_a=0
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(labels_earlier)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
        checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
                                     save_weights_only = True)

        for epoch in range(epochs):
            print("Epoch: %d" %(epoch+1))
            model.fit_generator(
                generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                          labels2Idx= labels2Idx,tokenize= tokenize), 
                steps_per_epoch= steps_per_epoch, epochs=1,
                validation_data = generator(filename ='/home/jindal/notebooks/twitter_data/twitter_classes_500k_dev.csv', 
                                            batch_size = batch_size, check = check_for_generator, 
                                           labels2Idx = labels2Idx, tokenize = tokenize),
                validation_steps = math.ceil(len(dev_labels) / batch_size),
                callbacks = [checkpoint]
            )

#             testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
#             for i in range(len(dev_sentences)):
#                 testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
#             results = model.predict(testset_features)


# #             idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
#             predLabels = results.argmax(axis=-1)
#             devLabels = devLabels
#             f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
#             r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
#             p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
#             a = accuracy_score(devLabels, predLabels)
#             if max_f1 < f1:
#                 print("model saved. F1 is %f" %(f1))
#                 model.save(filename_to_save_weigths)
#                 max_f1 = f1
#                 max_p = p
#                 max_r = r
#                 max_a = a
#             text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
#             print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
#         to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
#         print(to_write)
#         f.write(to_write)
#         total_f1+=max_f1
#         total_prec+=max_p
#         total_acc+=max_a
#         total_recall+=max_r    
#         print("*****************************************************************************")
#     final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
#     print(final_text)
#     f.write(final_text)
#     f.close()




SyntaxError: invalid syntax (<ipython-input-8-108820a7d5bd>, line 8)

In [86]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 1
number_of_epochs = 10
twitterclasses_pretraining_log = '/home/jindal/notebooks/jindal/NER/language_model/results_pretraining_twitterclasses.txt' 
twitterclasses_pretraining_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_twitterclasses.h5'
batch_size=32*4
twitterclasses_train_file='/home/jindal/notebooks/twitter_data/twitter_classes_500k_train.csv'
tokenize = False
labels2Idx = labels2Id
f1_measure='macro'
pos_label=0
load_model_weights=False
# model_weights_file:'give filepath as str'=None, 
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=2



In [87]:
len(labels2Idx)

500

In [88]:
test_model(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels,
           number_of_tests= number_of_tests,
          number_of_epochs=number_of_epochs,
           filename_to_log=twitterclasses_pretraining_log, 
           filename_to_save_weigths=twitterclasses_pretraining_save_weigths,
           tokenize= tokenize, 
           labels2Idx= labels2Idx,
          batch_size=batch_size,
           train_file=twitterclasses_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
          nb_sequence_length=nb_sequence_length,
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

Test 1/1
Epoch: 1
Epoch 1/1
3125/3125 [==============================] - 780s 250ms/step - loss: 4.1036 - acc: 0.2424 - val_loss: 3.8253 - val_acc: 0.2802
Epoch: 2
Epoch 1/1
3125/3125 [==============================] - 810s 259ms/step - loss: 3.7542 - acc: 0.2989 - val_loss: 3.6883 - val_acc: 0.3025
Epoch: 3
Epoch 1/1
3125/3125 [==============================] - 815s 261ms/step - loss: 3.6423 - acc: 0.3167 - val_loss: 3.6047 - val_acc: 0.3182
Epoch: 4
Epoch 1/1
3125/3125 [==============================] - 811s 260ms/step - loss: 3.5797 - acc: 0.3264 - val_loss: 3.5725 - val_acc: 0.3198
Epoch: 5
Epoch 1/1
3125/3125 [==============================] - 821s 263ms/step - loss: 3.5343 - acc: 0.3336 - val_loss: 3.5454 - val_acc: 0.3239
Epoch: 6
Epoch 1/1
3125/3125 [==============================] - 793s 254ms/step - loss: 3.4973 - acc: 0.3394 - val_loss: 3.5259 - val_acc: 0.3284
Epoch: 7
Epoch 1/1
3125/3125 [==============================] - 785s 251ms/step - loss: 3.4718 - acc: 0.3430 - val_

# TRANSFER LEARNING HERE


In [37]:
def test_model_tl(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               filename_to_save_weigths,
               batch_size, 
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None,
                ):
    
    f = open(filename_to_log,"w")
    
   
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(1000)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)
                print("removing top layer")
                model.layers.pop()
                output = Dense(2, activation = 'softmax')(model.layers[-1].output)
                final_model = Model(inputs=model.input, outputs=[output])
                final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
#                 for layer in final_model.layers:
#                     print(layer.name)

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
#         checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
#                                      save_weights_only = True)


        max_f1=0
        max_p=0
        max_r=0
        max_a=0
        for epoch in range(epochs):
            print("Epoch: %d" %(epoch+1))
            final_model.fit_generator(
                generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                          labels2Idx= labels2Idx,tokenize= tokenize), 
                steps_per_epoch= steps_per_epoch, epochs=1,
#                 validation_data = generator(filename ='/home/jindal/notebooks/twitter_data/twitter_classes_500k_dev.csv', 
#                                             batch_size = batch_size, check = check_for_generator, 
#                                            labels2Idx = labels2Idx, tokenize = tokenize),
#                 validation_steps = math.ceil(len(dev_labels) / batch_size),
#                 callbacks = [checkpoint]
            )

            testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
            for i in range(len(dev_sentences)):
                testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
            results = final_model.predict(testset_features)


#             idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
            predLabels = results.argmax(axis=-1)
            devLabels = devLabels
            f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
            r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            a = accuracy_score(devLabels, predLabels)
            if max_f1 < f1:
                print("model saved. F1 is %f" %(f1))
                final_model.save(filename_to_save_weigths)
                max_f1 = f1
                max_p = p
                max_r = r
                max_a = a
            text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
            print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
        to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
        print(to_write)
        f.write(to_write)
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    f.write(final_text)
    f.close()




In [26]:
n_labels =2

In [27]:
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(filetrain='/home/gwiedemann/notebooks/OffLang/sample_train.txt',
                   filedev='/home/gwiedemann/notebooks/OffLang/sample_dev.txt', check=3)

In [28]:
print(dev_sentences[0])
print(dev_labels[:20])

@FilmElf_yt Diese ganzen leute die du angesprochen hast sind untermenschen ja aber das alte Deutschland und deutsche volk waren ehrenwerte leute und ein stolzes Land
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]


In [38]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 5
number_of_epochs = 50
twitterclasses_tl_log = '/home/jindal/notebooks/jindal/NER/language_model/results_tl_twitterclasses7m_1000.txt' 
twitterclasses_tl_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_twitterclasses7m_1000.h5'
batch_size=32
twitterclasses_tl_train_file='/home/gwiedemann/notebooks/OffLang/sample_train.txt'
f1_measure='binary'
pos_label=1
load_model_weights=True
model_weights_file = '/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_twitterclasses7m_1000.h5'
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=3

In [ ]:
test_model_tl(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels, 
           number_of_tests= number_of_tests,
           number_of_epochs=number_of_epochs, 
           filename_to_log=twitterclasses_tl_log, 
           filename_to_save_weigths=twitterclasses_tl_save_weigths,
           batch_size=batch_size,
           train_file=twitterclasses_tl_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
           model_weights_file = model_weights_file, 
           nb_sequence_length=nb_sequence_length, 
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

Test 1/5
removing top layer
Epoch: 1
Epoch 1/1
132/132 [==============================] - 35s 263ms/step - loss: 0.5934 - acc: 0.6927
model saved. F1 is 0.644000
Test-Data: Prec: 0.691, Rec: 0.603, F1: 0.644, Acc: 0.780
Epoch: 2
Epoch 1/1
132/132 [==============================] - 30s 231ms/step - loss: 0.4883 - acc: 0.7640
model saved. F1 is 0.662698
Test-Data: Prec: 0.705, Rec: 0.625, F1: 0.663, Acc: 0.790
Epoch: 3
Epoch 1/1
132/132 [==============================] - 31s 233ms/step - loss: 0.4513 - acc: 0.7796
Test-Data: Prec: 0.707, Rec: 0.614, F1: 0.657, Acc: 0.788
Epoch: 4
Epoch 1/1
132/132 [==============================] - 30s 229ms/step - loss: 0.4044 - acc: 0.8078
model saved. F1 is 0.672098
Test-Data: Prec: 0.737, Rec: 0.618, F1: 0.672, Acc: 0.801
Epoch: 5
Epoch 1/1
132/132 [==============================] - 30s 229ms/step - loss: 0.3860 - acc: 0.8203
model saved. F1 is 0.677104
Test-Data: Prec: 0.709, Rec: 0.648, F1: 0.677, Acc: 0.796
Epoch: 6
Epoch 1/1
132/132 [============

# gradual unfreezing


In [52]:
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(filetrain='/home/gwiedemann/notebooks/OffLang/sample_train.txt',
                   filedev='/home/gwiedemann/notebooks/OffLang/sample_dev.txt', check=3)

In [55]:
print(dev_sentences[0])
print(len(dev_sentences))
print(len(dev_labels))

@FilmElf_yt Diese ganzen leute die du angesprochen hast sind untermenschen ja aber das alte Deutschland und deutsche volk waren ehrenwerte leute und ein stolzes Land
808
808


In [84]:
# list of tuples. Every tuple contains range of layers which need to be unfrozen. Rest all are frozen
single_unfreeze_bottom_up = [(18, 18), (17, 16), (15, 3), (2, 1), (18,1)] 
single_unfreeze_top_down = [(18, 18),   (2, 1),(15, 3), (17, 16), (18,1)]
all_unfreeze = [(18,1)]
gradual_unfreezing = [(18,18), (18,16), (18,3), (18,1)]

# strings =['suf_bu', 'suf_td','all_unfreeze','gradual_unfreeze']
strings=['gradual_unfreeze']
# unfreeze_strategy = [single_unfreeze_bottom_up, single_unfreeze_top_down, all_unfreeze, gradual_unfreezing]
unfreeze_strategy = [gradual_unfreezing]

In [85]:
def test_model_tl_unfreezing(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels2Idx,
               filename_to_save_weigths,
               batch_size, 
               unfreezing_strategy: 'list containing a tuple of indices to unfreeze at each step',
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None):
    
    f = open(filename_to_log, 'w', encoding='utf-8')
    f.close()
   
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(1000)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)
                print("removing top layer")
                model.layers.pop()
                output = Dense(2, activation = 'softmax')(model.layers[-1].output)
                final_model = Model(inputs=model.input, outputs=[output])

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
#         checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
#                                      save_weights_only = True)


        max_f1=0
        max_p=0
        max_r=0
        max_a=0
        
        # load pretrained weights
        # model.compile
        # save tmp weights
        # iterate over layers
        #    load tmp weights
        #    iterate over epochs
        #        unfreeze top frozen layer
        #        save best model as tmp weights
        
        
        final_model.save(filename_to_save_weigths)
        
        # layers_to_unfreeze = [18, 16, 3, 1]
        
        for ulayer in unfreezing_strategy:
            print("unfreezing " + final_model.layers[ulayer[0]].name)
            print("---------------------------------------")
            final_model.load_weights(filename_to_save_weigths)            
            for i, layer in enumerate(final_model.layers):
                
                # TF strategy: gradual unfreezing
                #if i >= ulayer:
                #    layer.trainable = True
                #else:
                #    layer.trainable = False
                # 
                ## TF strategy: single
                
                if i >= ulayer[1] and i <= ulayer[0]:
                    layer.trainable = True
                else:
                    layer.trainable = False
                    
                print(str(i) + ' ' + layer.name + ' ' + str(layer.trainable))
            final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
        
            for epoch in range(epochs):
                print("Epoch: %d/%d" %(epoch+1, epochs))
                final_model.fit_generator(
                    generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                              labels2Idx= labels2Idx,tokenize= tokenize), 
                    steps_per_epoch= steps_per_epoch, epochs=1
                )

                testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
                for i in range(len(dev_sentences)):
                    testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
                results = final_model.predict(testset_features)

                predLabels = results.argmax(axis=-1)
                devLabels = devLabels
                f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
                r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                a = accuracy_score(devLabels, predLabels)
                if max_f1 < f1:
                    print("model saved. F1 is %f" %(f1))
                    final_model.save(filename_to_save_weigths)
                    max_f1 = f1
                    max_p = p
                    max_r = r
                    max_a = a
                text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
                print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
        to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
        print(to_write)
        with open(filename_to_log,'a') as f:
            f.write(to_write)
#         f.close()
#         f = open(filename_to_log, 'a', encoding='utf-8')
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    with open(filename_to_log,'a') as f:
        f.write(final_text)
#     f.close()

In [90]:
# list of tuples. Every tuple contains range of layers which need to be unfrozen. Rest all are frozen
single_unfreeze_bottom_up = [(18, 18), (17, 16), (15, 3), (2, 1), (18,1)] 
single_unfreeze_top_down = [(18, 18),   (2, 1),(15, 3), (17, 16), (18,1)]
all_unfreeze = [(18,1)]
gradual_unfreezing = [(18,18), (18,16), (18,3), (18,1)]

# strings =['suf_bu', 'suf_td','all_unfreeze','gradual_unfreeze']
strings=['gradual_unfreeze']
# unfreeze_strategy = [single_unfreeze_bottom_up, single_unfreeze_top_down, all_unfreeze, gradual_unfreezing]
unfreeze_strategy = [gradual_unfreezing]

In [ ]:
for i in range(len(strings)):
    string = strings[i]
    print("approach: %s" %(string))
    
    generator = sequential_generator
    train_sentences = train_sentences
    devLabels = dev_labels
    number_of_tests = 5
    number_of_epochs = 50
    labels2Id = labels2Idx
    twitterclasses_tl_log = '/home/jindal/notebooks/jindal/NER/language_model/results_tl_twitterclasses7m_1000_' +string+'.txt' 
    print("log file: %s" %(twitterclasses_tl_log))
    twitterclasses_tl_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_twitterclasses7m_1000_'+string+'.h5'
    print("save weights file: %s" %(twitterclasses_tl_save_weigths))
    batch_size=32
    twitterclasses_tl_train_file='/home/gwiedemann/notebooks/OffLang/sample_train.txt'
    f1_measure='binary'
    pos_label=1
    strategy = unfreeze_strategy[i]
    print(strategy)
    load_model_weights=True
    model_weights_file = '/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_twitterclasses7m_1000.h5'
    nb_sequence_length = nb_sequence_length
    nb_embedding_dims= nb_embedding_dims
    check_for_generator=3
    
    test_model_tl_unfreezing(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels, 
           number_of_tests= number_of_tests,
           number_of_epochs=number_of_epochs, 
           filename_to_log=twitterclasses_tl_log, 
           labels2Idx = labels2Id,
           filename_to_save_weigths=twitterclasses_tl_save_weigths,
           batch_size=batch_size,
           unfreezing_strategy = strategy,       
           train_file=twitterclasses_tl_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
           model_weights_file = model_weights_file, 
           nb_sequence_length=nb_sequence_length, 
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

approach: gradual_unfreeze
log file: /home/jindal/notebooks/jindal/NER/language_model/results_tl_twitterclasses7m_1000_gradual_unfreeze.txt
save weights file: /home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_twitterclasses7m_1000_gradual_unfreeze.h5
[(18, 18), (18, 16), (18, 3), (18, 1)]
Test 1/5
removing top layer
unfreezing dense_205
---------------------------------------
0 input_70 False
1 bidirectional_70 False
2 leaky_re_lu_346 False
3 conv1d_208 False
4 conv1d_209 False
5 conv1d_210 False
6 leaky_re_lu_347 False
7 leaky_re_lu_348 False
8 leaky_re_lu_349 False
9 global_max_pooling1d_208 False
10 global_max_pooling1d_209 False
11 global_max_pooling1d_210 False
12 dropout_208 False
13 dropout_209 False
14 dropout_210 False
15 concatenate_70 False
16 dense_203 False
17 leaky_re_lu_350 False
18 dense_205 True
Epoch: 1/50
Epoch 1/1
132/132 [==============================] - 26s 197ms/step - loss: 0.6738 - acc: 0.6456
model saved. F1 is 0.470588
Test-Data: Prec:

132/132 [==============================] - 10s 74ms/step - loss: 0.5401 - acc: 0.7128
Test-Data: Prec: 0.669, Rec: 0.431, F1: 0.524, Acc: 0.741
Epoch: 45/50
Epoch 1/1
132/132 [==============================] - 10s 75ms/step - loss: 0.5428 - acc: 0.7287
Test-Data: Prec: 0.674, Rec: 0.479, F1: 0.560, Acc: 0.751
Epoch: 46/50
Epoch 1/1
132/132 [==============================] - 10s 74ms/step - loss: 0.5380 - acc: 0.7202
Test-Data: Prec: 0.650, Rec: 0.479, F1: 0.552, Acc: 0.743
Epoch: 47/50
Epoch 1/1
132/132 [==============================] - 10s 74ms/step - loss: 0.5399 - acc: 0.7202
Test-Data: Prec: 0.649, Rec: 0.464, F1: 0.541, Acc: 0.740
Epoch: 48/50
Epoch 1/1
132/132 [==============================] - 10s 73ms/step - loss: 0.5444 - acc: 0.7195
Test-Data: Prec: 0.663, Rec: 0.442, F1: 0.530, Acc: 0.741
Epoch: 49/50
Epoch 1/1
132/132 [==============================] - 10s 75ms/step - loss: 0.5334 - acc: 0.7221
Test-Data: Prec: 0.647, Rec: 0.487, F1: 0.556, Acc: 0.743
Epoch: 50/50
Epoch 1/

132/132 [==============================] - 10s 76ms/step - loss: 0.4684 - acc: 0.7623
Test-Data: Prec: 0.650, Rec: 0.648, F1: 0.649, Acc: 0.769
Epoch: 40/50
Epoch 1/1
132/132 [==============================] - 10s 74ms/step - loss: 0.4789 - acc: 0.7618
Test-Data: Prec: 0.695, Rec: 0.614, F1: 0.652, Acc: 0.783
Epoch: 41/50
Epoch 1/1
132/132 [==============================] - 10s 75ms/step - loss: 0.4752 - acc: 0.7609
Test-Data: Prec: 0.683, Rec: 0.614, F1: 0.647, Acc: 0.778
Epoch: 42/50
Epoch 1/1
132/132 [==============================] - 10s 74ms/step - loss: 0.4705 - acc: 0.7675
Test-Data: Prec: 0.668, Rec: 0.633, F1: 0.650, Acc: 0.775
Epoch: 43/50
Epoch 1/1
132/132 [==============================] - 10s 76ms/step - loss: 0.4718 - acc: 0.7670
Test-Data: Prec: 0.678, Rec: 0.584, F1: 0.628, Acc: 0.771
Epoch: 44/50
Epoch 1/1
132/132 [==============================] - 10s 74ms/step - loss: 0.4614 - acc: 0.7798
Test-Data: Prec: 0.661, Rec: 0.599, F1: 0.629, Acc: 0.766
Epoch: 45/50
Epoch 1/

132/132 [==============================] - 10s 77ms/step - loss: 0.1133 - acc: 0.9583
Test-Data: Prec: 0.684, Rec: 0.655, F1: 0.669, Acc: 0.786
Epoch: 35/50
Epoch 1/1
132/132 [==============================] - 10s 78ms/step - loss: 0.1061 - acc: 0.9586
Test-Data: Prec: 0.709, Rec: 0.592, F1: 0.645, Acc: 0.785
Epoch: 36/50
Epoch 1/1
132/132 [==============================] - 10s 76ms/step - loss: 0.1139 - acc: 0.9529
Test-Data: Prec: 0.681, Rec: 0.655, F1: 0.668, Acc: 0.785
Epoch: 37/50
Epoch 1/1
132/132 [==============================] - 10s 75ms/step - loss: 0.1073 - acc: 0.9624
Test-Data: Prec: 0.670, Rec: 0.685, F1: 0.678, Acc: 0.785
Epoch: 38/50
Epoch 1/1
132/132 [==============================] - 10s 76ms/step - loss: 0.1085 - acc: 0.9588
Test-Data: Prec: 0.710, Rec: 0.614, F1: 0.659, Acc: 0.790
Epoch: 39/50
Epoch 1/1
132/132 [==============================] - 10s 76ms/step - loss: 0.1043 - acc: 0.9574
Test-Data: Prec: 0.699, Rec: 0.599, F1: 0.645, Acc: 0.782
Epoch: 40/50
Epoch 1/

132/132 [==============================] - 28s 211ms/step - loss: 0.0494 - acc: 0.9853
Test-Data: Prec: 0.717, Rec: 0.607, F1: 0.657, Acc: 0.791
Epoch: 31/50
Epoch 1/1
132/132 [==============================] - 28s 213ms/step - loss: 0.0548 - acc: 0.9813
Test-Data: Prec: 0.697, Rec: 0.622, F1: 0.657, Acc: 0.786
Epoch: 32/50
Epoch 1/1
132/132 [==============================] - 29s 220ms/step - loss: 0.0449 - acc: 0.9825
Test-Data: Prec: 0.679, Rec: 0.674, F1: 0.677, Acc: 0.787
Epoch: 33/50
Epoch 1/1
132/132 [==============================] - 28s 214ms/step - loss: 0.0439 - acc: 0.9839
Test-Data: Prec: 0.686, Rec: 0.712, F1: 0.699, Acc: 0.797
Epoch: 34/50
Epoch 1/1
132/132 [==============================] - 29s 223ms/step - loss: 0.0371 - acc: 0.9875
Test-Data: Prec: 0.689, Rec: 0.655, F1: 0.672, Acc: 0.788
Epoch: 35/50
Epoch 1/1
132/132 [==============================] - 28s 214ms/step - loss: 0.0544 - acc: 0.9820
Test-Data: Prec: 0.712, Rec: 0.629, F1: 0.668, Acc: 0.793
Epoch: 39/50
Ep

...

132/132 [==============================] - 28s 213ms/step - loss: 0.0759 - acc: 0.9723
Test-Data: Prec: 0.692, Rec: 0.607, F1: 0.647, Acc: 0.781
Epoch: 14/50
Epoch 1/1
132/132 [==============================] - 28s 215ms/step - loss: 0.0684 - acc: 0.9721
Test-Data: Prec: 0.675, Rec: 0.693, F1: 0.684, Acc: 0.788
Epoch: 15/50
Epoch 1/1
132/132 [==============================] - 28s 212ms/step - loss: 0.0720 - acc: 0.9744
Test-Data: Prec: 0.681, Rec: 0.689, F1: 0.685, Acc: 0.791
Epoch: 16/50
Epoch 1/1
132/132 [==============================] - 28s 214ms/step - loss: 0.0768 - acc: 0.9716
Test-Data: Prec: 0.669, Rec: 0.727, F1: 0.697, Acc: 0.791
Epoch: 17/50
Epoch 1/1
132/132 [==============================] - 28s 215ms/step - loss: 0.0579 - acc: 0.9796
Test-Data: Prec: 0.679, Rec: 0.689, F1: 0.684, Acc: 0.790
Epoch: 18/50
Epoch 1/1
132/132 [==============================] - 28s 214ms/step - loss: 0.0620 - acc: 0.9756
Test-Data: Prec: 0.701, Rec: 0.659, F1: 0.680, Acc: 0.795
Epoch: 19/50
Ep

132/132 [==============================] - 10s 74ms/step - loss: 0.5500 - acc: 0.7093
Test-Data: Prec: 0.690, Rec: 0.434, F1: 0.533, Acc: 0.749
Epoch: 8/50
Epoch 1/1
132/132 [==============================] - 10s 74ms/step - loss: 0.5484 - acc: 0.7169
model saved. F1 is 0.557823
Test-Data: Prec: 0.707, Rec: 0.461, F1: 0.558, Acc: 0.759
Epoch: 9/50
Epoch 1/1
132/132 [==============================] - 10s 75ms/step - loss: 0.5412 - acc: 0.7247
Test-Data: Prec: 0.719, Rec: 0.449, F1: 0.553, Acc: 0.760
Epoch: 10/50
Epoch 1/1
132/132 [==============================] - 10s 76ms/step - loss: 0.5408 - acc: 0.7225
Test-Data: Prec: 0.684, Rec: 0.453, F1: 0.545, Acc: 0.750
Epoch: 11/50
Epoch 1/1
132/132 [==============================] - 10s 74ms/step - loss: 0.5445 - acc: 0.7209
model saved. F1 is 0.560345
Test-Data: Prec: 0.660, Rec: 0.487, F1: 0.560, Acc: 0.748
Epoch: 12/50
Epoch 1/1
132/132 [==============================] - 10s 75ms/step - loss: 0.5402 - acc: 0.7209
Test-Data: Prec: 0.678, R

132/132 [==============================] - 10s 76ms/step - loss: 0.5275 - acc: 0.7313
model saved. F1 is 0.582121
Test-Data: Prec: 0.654, Rec: 0.524, F1: 0.582, Acc: 0.751
Epoch: 4/50
Epoch 1/1
132/132 [==============================] - 10s 75ms/step - loss: 0.5212 - acc: 0.7322
Test-Data: Prec: 0.677, Rec: 0.502, F1: 0.576, Acc: 0.756
Epoch: 5/50
Epoch 1/1
132/132 [==============================] - 10s 76ms/step - loss: 0.5177 - acc: 0.7403
model saved. F1 is 0.586207
Test-Data: Prec: 0.690, Rec: 0.509, F1: 0.586, Acc: 0.762
Epoch: 6/50
Epoch 1/1
132/132 [==============================] - 10s 75ms/step - loss: 0.5123 - acc: 0.7379
model saved. F1 is 0.593684
Test-Data: Prec: 0.678, Rec: 0.528, F1: 0.594, Acc: 0.761
Epoch: 7/50
Epoch 1/1
132/132 [==============================] - 10s 75ms/step - loss: 0.5175 - acc: 0.7337
model saved. F1 is 0.602151
Test-Data: Prec: 0.707, Rec: 0.524, F1: 0.602, Acc: 0.771
Epoch: 8/50
Epoch 1/1
132/132 [==============================] - 10s 75ms/step -

132/132 [==============================] - 10s 76ms/step - loss: 0.4621 - acc: 0.7768
Test-Data: Prec: 0.660, Rec: 0.588, F1: 0.622, Acc: 0.764
unfreezing dense_211
---------------------------------------
0 input_72 False
1 bidirectional_72 False
2 leaky_re_lu_356 False
3 conv1d_214 True
4 conv1d_215 True
5 conv1d_216 True
6 leaky_re_lu_357 True
7 leaky_re_lu_358 True
8 leaky_re_lu_359 True
9 global_max_pooling1d_214 True
10 global_max_pooling1d_215 True
11 global_max_pooling1d_216 True
12 dropout_214 True
13 dropout_215 True
14 dropout_216 True
15 concatenate_72 True
16 dense_209 True
17 leaky_re_lu_360 True
18 dense_211 True
Epoch: 1/50
Epoch 1/1
132/132 [==============================] - 30s 224ms/step - loss: 0.4679 - acc: 0.7663
Test-Data: Prec: 0.660, Rec: 0.596, F1: 0.626, Acc: 0.765
Epoch: 2/50
Epoch 1/1
132/132 [==============================] - 10s 77ms/step - loss: 0.4405 - acc: 0.7905
Test-Data: Prec: 0.659, Rec: 0.644, F1: 0.652, Acc: 0.772
Epoch: 3/50
Epoch 1/1
132/132 [=

Test-Data: Prec: 0.674, Rec: 0.652, F1: 0.663, Acc: 0.781
Epoch: 46/50
Epoch 1/1
132/132 [==============================] - 10s 77ms/step - loss: 0.0916 - acc: 0.9659
Test-Data: Prec: 0.691, Rec: 0.603, F1: 0.644, Acc: 0.780
Epoch: 47/50
Epoch 1/1
132/132 [==============================] - 10s 78ms/step - loss: 0.0906 - acc: 0.9669
Test-Data: Prec: 0.648, Rec: 0.663, F1: 0.656, Acc: 0.770
Epoch: 48/50
Epoch 1/1
132/132 [==============================] - 10s 75ms/step - loss: 0.0931 - acc: 0.9657
Test-Data: Prec: 0.664, Rec: 0.674, F1: 0.669, Acc: 0.780
Epoch: 49/50
Epoch 1/1
132/132 [==============================] - 10s 77ms/step - loss: 0.0810 - acc: 0.9678
Test-Data: Prec: 0.659, Rec: 0.659, F1: 0.659, Acc: 0.775
Epoch: 50/50
Epoch 1/1
132/132 [==============================] - 10s 76ms/step - loss: 0.0920 - acc: 0.9633
Test-Data: Prec: 0.649, Rec: 0.685, F1: 0.667, Acc: 0.774
unfreezing dense_211
---------------------------------------
0 input_72 False
1 bidirectional_72 True
2 lea